In [ ]:
from datetime import datetime

import metpy.calc as mpcalc
from metpy.plots import declarative
from metpy.units import units
import numpy as np
from siphon.catalog import TDSCatalog
import xarray as xr

In [ ]:
# Get Satellite Data

# Construct the data_url string
data_url = ('https://thredds.ucar.edu/thredds/dodsC/casestudies/2019blizzard/goes/GOES16/CONUS/'
            'Channel09/20190313/GOES16_CONUS_20190313_000719_6.95_2km_30.1N_87.1W.nc4')

# Open most recent file available
sat_ds = xr.open_dataset(data_url)

# Apply a square root correction for visible imagery only
#ds = ds.map(np.sqrt, keep_attrs=True)

# Grab time from file and make into datetime object for plotting and later data access
sat_date = sat_ds.time.values.astype('datetime64[ms]').astype('O')

In [ ]:
# Get Model Data, set to a time using satellite time
date = datetime(sat_date.year, sat_date.month, sat_date.day, sat_date.hour)

# Remote access to the dataset from the NCDC site
catUrl = "https://rda.ucar.edu/thredds/catalog/aggregations/g/ds083.2/2/catalog.xml"
datasetName = "Full Collection Dataset"
catalog = TDSCatalog(catUrl)
dataset = catalog.datasets[datasetName]
ds = dataset.remote_access(use_xarray=True)

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-150, 360-55)).metpy.parse_cf()